In [1]:
!pip install tldextract
import pandas as pd
import numpy as np
import ipaddress as ip
from urllib.parse import urlparse
import tldextract

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 51kB 15.8MB/s eta 0:00:01


/opt/conda/envs/python35-paddle120-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


用于提取特征的函数：

In [2]:
def countdots(url):  
    return url.count('.')

def isip(uri):
    try:
        if ip.ip_address(uri):
            return 1
    except:
        return 0

def isPresentHyphen(url):
    return url.count('-')

def isPresentAt(url):
    return url.count('@')

def isPresentDSlash(url):
    return url.count('//')

def countSubDir(url):
    return url.count('/')

def get_ext(url):
    """Return the filename extension from url, or ''."""  
    root, ext = splitext(url)
    return ext

def countSubDomain(subdomain):
    if not subdomain:
        return 0
    else:
        return len(subdomain.split('.'))

def countQueries(query):
    if not query:
        return 0
    else:
        return len(query.split('&'))

In [3]:
def getFeatures(url, label): 
     #2016's top most suspicious TLD and words
    Suspicious_TLD=['zip','cricket','link','work','party','gq','kim','country','science','tk']
    Suspicious_Domain=['luckytime.co.kr','mattfoll.eu.interia.pl','trafficholder.com','dl.baixaki.com.br','bembed.redtube.comr','tags.expo9.exponential.com','deepspacer.com','funad.co.kr','trafficconverter.biz']
    #trend micro's top malicious domains 
    result = []
    url = str(url)
    
    #add the url to feature set
    result.append(url)
    result.append(str(label))
    
    #parse the URL and extract the domain information
    path = urlparse(url)
    ext = tldextract.extract(url)
    
    #counting number of dots in subdomain    
    result.append(countdots(ext.subdomain))
    
    #checking hyphen in domain   
    result.append(isPresentHyphen(path.netloc))
    
    #length of URL    
    result.append(len(url))
    
    #checking @ in the url    
    result.append(isPresentAt(path.netloc))
    
    #checking presence of double slash    
    result.append(isPresentDSlash(path.path))
    
    #Count number of subdir    
    result.append(countSubDir(path.path))
    
    #number of sub domain    
    result.append(countSubDomain(ext.subdomain))
    
    #length of domain name    
    result.append(len(path.netloc))
    
    #count number of queries    
    result.append(len(path.query))
    
    #Adding domain information
    
    #if IP address is being used as a URL     
    result.append(isip(ext.domain))
    
    #presence of Suspicious_TLD
    result.append(1 if ext.suffix in Suspicious_TLD else 0)
    
    #presence of suspicious domain
    result.append(1 if '.'.join(ext[1:]) in Suspicious_Domain else 0 )

    return result

# 读取原始数据：

In [4]:
data = pd.read_csv('data.txt')
len(data)

14157

目标数据集featureSet的各列名：

In [5]:
featureSet = pd.DataFrame(columns=('url','label','no of dots','presence of hyphen','len of url','presence of at',\
    'presence of double slash','no of subdir','no of subdomain','len of domain','no of queries','is IP','presence of Suspicious_TLD',\
    'presence of suspicious domain'))

将各个特征写入目标数据集featureSet

In [6]:
for i in range(len(data)):
        features = getFeatures(data["url"].loc[i], data['label'].loc[i])   
        featureSet.loc[i] = features   
        if i %1000 == 0:
            print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


将含有特征的数据集保存为trainData.txt

In [7]:
featureSet
featureSet.to_csv('trainData.txt')

# 训练模型

**决策树模型：**

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

从含有特征的数据文件中读取数据，x为输入的数据，y为输出的数据；
输入的数据需要把url、label两列去掉，除此之外还有第一列的序号需要去除；
输出的数据仅为label一列的值

In [27]:
data = pd.read_csv('trainData.txt')
x = data.drop(['url','label'],axis=1).values
x = np.delete(x,obj=0,axis=1)
y = data['label'].values

使用train_test_split将0.3的数据集划分为测试集

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y ,test_size=0.3)
x_train

array([[ 0,  0, 13, ...,  0,  0,  0],
       [ 0,  0, 15, ...,  0,  0,  0],
       [ 0,  0, 43, ...,  0,  0,  0],
       ...,
       [ 0,  0, 14, ...,  0,  0,  0],
       [ 0,  0, 10, ...,  0,  0,  0],
       [ 0,  0, 15, ...,  0,  0,  0]])

定义训练集为决策树模型；
使用.fit训练模型

In [29]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

**模型评估：**
使用.predict预测测试集的输出，用accuracy_score根据测试集输出和预测结果求准确率

In [30]:
predictions = classifier.predict(x_test)
accuracy = 100.0 * accuracy_score(y_test, predictions)
print(accuracy)

98.9406779661017


In [32]:
recall_score(y_test,predictions)

0.9177570093457944

In [31]:
precision_score(y_test,predictions)

0.9979674796747967

In [14]:
score_train = classifier.score(x_train,y_train)
print(score_train)

0.9905136744373801


**随机森林：**

In [15]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
acc_random_forest

/opt/conda/envs/python35-paddle120-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


99.05

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
y_pred = random_forest.predict(x_test)
accuracy_score(y_test, y_pred)

0.9912900188323918

In [17]:
precision_score(y_test, y_pred)

1.0

In [18]:
recall_score(y_test, y_pred)

0.9288461538461539

In [19]:
def get_url_features(url): 
     #2016's top most suspicious TLD and words
    Suspicious_TLD=['zip','cricket','link','work','party','gq','kim','country','science','tk']
    Suspicious_Domain=['luckytime.co.kr','mattfoll.eu.interia.pl','trafficholder.com','dl.baixaki.com.br','bembed.redtube.comr','tags.expo9.exponential.com','deepspacer.com','funad.co.kr','trafficconverter.biz']
    
    #trend micro's top malicious domains 
    url = str(url)
        
    #parse the URL and extract the domain information
    path = urlparse(url)
    ext = tldextract.extract(url)

    return [countdots(ext.subdomain),isPresentHyphen(path.netloc), len(url), isPresentAt(path.netloc), isPresentDSlash(path.path), countSubDir(path.path),\
            countSubDomain(ext.subdomain), len(path.netloc), len(path.query), isip(ext.domain), 1 if ext.suffix in Suspicious_TLD else 0,\
            1 if '.'.join(ext[1:]) in Suspicious_Domain else 0]


In [20]:
def whether_good(test_url):
    test_features=[]
    test_features.append(get_url_features(test_url))
    # print(test_features)
    pre_random = random_forest.predict(test_features)
    pre_tree = classifier.predict(test_features)
    print('random forest',pre_random)
    print('classify tree',pre_tree)

In [21]:
test_url = 'zhihu.com'

whether_good(test_url)

random forest [0]
classify tree [0]


In [22]:
whether_good('destre45.com/p/f.exe')

random forest [1]
classify tree [1]


In [23]:
whether_good('baidu.com')

random forest [0]
classify tree [0]


In [24]:
whether_good('www.archigate.it/')

random forest [1]
classify tree [1]


In [25]:
whether_good('sina.cn')

random forest [0]
classify tree [0]
